In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
from sklearn import linear_model
from sklearn import preprocessing

D:\annoconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [53]:
raw_train_df = train_df.copy()
raw_test_df = test_df.copy()

In [54]:
print(len(test_df.columns))
print(len(train_df.columns))
print('需要处理的非空给的column的个数是'+str((train_df.isnull().sum().values!=0).sum())) 
#这样可以看看 是不是所有的空的行都被处理了 pd.isnull后是DataFrame sum后是Series
pd.set_option('display.max_rows', 50)  #设置显示的最大行数
train_df.info()

80
81
需要处理的非空给的column的个数是19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object

In [55]:
test_df.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
                 ... 
Functional          2
Fireplaces          0
FireplaceQu       730
GarageType         76
GarageYrBlt        78
GarageFinish       78
GarageCars          1
GarageArea          1
GarageQual         78
GarageCond         78
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
PoolQC           1456
Fence            1169
MiscFeatur

In [56]:
discard_columns = ['Alley', 'PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu']
train_mean_value_list = ['LotFrontage', 'MasVnrArea','GarageYrBlt']
train_object_list = ['MasVnrType', 'BsmtQual', 'BsmtCond', 'Electrical', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2','GarageType','GarageFinish','GarageQual','GarageCond']

In [57]:
#######后加的  一开始是以train集中为空的column为准，发现漏了好多，好多train不空，test中为空，出现问题#############
#上面是直接眼看的，这次改进了！！！
test_null_list = test_df.isnull().sum().index[(test_df.isnull().sum().values)!=0].tolist()
print(len(test_null_list))
test_null_list

33


['MSZoning',
 'LotFrontage',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [58]:
a = np.array([1,2,3]) #list不支持bool值读取
print(a)
print(set(a))
a[[True, False, True]]

[1 2 3]
{1, 2, 3}


array([1, 3])

In [59]:
#把需要处理的合并成一个列表，包括train和test
temp_list = []
for i in test_null_list:
    print(test_df[i].dtype)
    temp_list.append(test_df[i].dtype)
print(np.array(temp_list) == 'object')
test_object_list = list(set(np.array(test_null_list)[np.array(temp_list) == 'object'])-set(discard_columns))
len(test_object_list)

object
float64
object
object
object
object
object
float64
object
object
object
object
float64
object
float64
float64
float64
float64
float64
object
object
object
object
float64
object
float64
float64
object
object
object
object
object
object
[ True False  True  True  True  True  True False  True  True  True  True
 False  True False False False False False  True  True  True  True False
  True False False  True  True  True  True  True  True]


17

In [60]:
test_mean_list = list(set(test_null_list)-set(test_object_list)-set(discard_columns))
print(test_object_list)
len(test_mean_list)

['Functional', 'Exterior2nd', 'SaleType', 'BsmtCond', 'BsmtExposure', 'BsmtQual', 'Utilities', 'BsmtFinType1', 'MasVnrType', 'GarageFinish', 'MSZoning', 'BsmtFinType2', 'GarageCond', 'Exterior1st', 'GarageType', 'GarageQual', 'KitchenQual']


11

In [61]:
for i in test_object_list:
    if i not in train_object_list:
        train_object_list.append(i)
for i in test_mean_list:
    if i not in train_mean_value_list:
        train_mean_value_list.append(i)
print(len(train_object_list))
print(len(train_mean_value_list))
train_object_list

18
11


['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'Electrical',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'Functional',
 'Exterior2nd',
 'SaleType',
 'Utilities',
 'MSZoning',
 'Exterior1st',
 'KitchenQual']

In [62]:
train_test = [train_df, test_df]#放到一个列表中用循环一起处理
for df in train_test: 
    df.drop(discard_columns, axis=1, inplace = True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non

In [63]:
for column in train_mean_value_list:
    for df in train_test:
        df[column].fillna(value=df[column].mean(), inplace=True)

for column in train_object_list:
    for df in train_test:
        df[column].fillna(df[column].mode()[0], inplace=True)
print(test_df.isnull().sum())
train_df.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
                ..
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 75, dtype: int64


Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
                ..
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Fireplaces       0
GarageType       0
GarageYrBlt      0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 76, dtype: int64

In [64]:
a = np.array([[1,2,3]])
b = a.copy()
c = b.copy()
c[0] = 0
print(b.all())
(a==b).all(keepdims=True)
a = np.array([[True, True]])
b = np.array([[True, False]])
np.logical_or(a, b)

True


array([[ True,  True]], dtype=bool)

In [65]:
##################对数值型的做transform##########


print(train_df['Street'].mode())#mode 是众数
print(train_df.dtypes)
a = np.array([[1]])

print(np.logical_or((train_df.dtypes.values=='float64'),(train_df.dtypes.values=='int64')))
number_column_index = np.logical_or((train_df.dtypes.values=='float64'),(train_df.dtypes.values=='int64'))
number_columns = train_df.columns[number_column_index].tolist()

str_columns_index = train_df.dtypes.values=='object' #bool类型 为了下面取索引的
str_columns = train_df.columns[str_columns_index].tolist()


print(str_columns_index)
print(len(number_columns))
print(len(train_df.columns))
len(str_columns)
print(str_columns)
print(number_columns)
# print(train_df.columns)
number_columns[0:2]

0    Pave
dtype: object
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
                  ...   
KitchenAbvGr       int64
KitchenQual       object
TotRmsAbvGrd       int64
Functional        object
Fireplaces         int64
GarageType        object
GarageYrBlt      float64
GarageFinish      object
GarageCars         int64
GarageArea         int64
GarageQual        object
GarageCond        object
PavedDrive        object
W

['Id', 'MSSubClass']

In [66]:
train_df.Street.value_counts()

Pave    1454
Grvl       6
Name: Street, dtype: int64

In [67]:
print(test_df['BsmtFinSF1'].dtype)  #发现train中的这个是int， test中是float
print(train_df['BsmtFinSF1'].dtype)   


test_df['BsmtFinSF1'].astype('int64')
print(train_df['BsmtFinSF1'].dtype) 

float64
int64
int64


In [68]:

##############这是直接把所有特征一起放进去变换的下面是每次放一个特征用循环，这个得到是array再转成DataFrame

scaler2 = preprocessing.StandardScaler()

scaler_para2 = scaler2.fit(train_df[number_columns[1:-1]])
array_number = scaler2.fit_transform(train_df[number_columns[1:-1]], scaler_para2)
pd.DataFrame(array_number, index=train_df.Id, columns=[ x + '_scaled' for x in number_columns[1:-1]])

########################################################################################################

,MSSubClass_scaled,LotFrontage_scaled,LotArea_scaled,OverallQual_scaled,OverallCond_scaled,YearBuilt_scaled,YearRemodAdd_scaled,MasVnrArea_scaled,BsmtFinSF1_scaled,BsmtFinSF2_scaled,BsmtUnfSF_scaled,TotalBsmtSF_scaled,1stFlrSF_scaled,2ndFlrSF_scaled,LowQualFinSF_scaled,GrLivArea_scaled,BsmtFullBath_scaled,BsmtHalfBath_scaled,FullBath_scaled,HalfBath_scaled,BedroomAbvGr_scaled,KitchenAbvGr_scaled,TotRmsAbvGrd_scaled,Fireplaces_scaled,GarageYrBlt_scaled,GarageCars_scaled,GarageArea_scaled,WoodDeckSF_scaled,OpenPorchSF_scaled,EnclosedPorch_scaled,3SsnPorch_scaled,ScreenPorch_scaled,PoolArea_scaled,MiscVal_scaled,MoSold_scaled,YrSold_scaled
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.073375,-2.293718e-01,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,-0.944591,-0.459303,-0.793434,1.161852,-0.120242,0.370333,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.912210,-0.951226,1.021157,0.311725,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
2,-0.872563,4.519361e-01,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,-0.641228,0.466465,0.257140,-0.795163,-0.120242,-0.482512,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.318683,0.600495,-0.104483,0.311725,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
3,0.073375,-9.311018e-02,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,-0.301643,-0.313369,-0.627826,1.189351,-0.120242,0.515013,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,-0.318683,0.600495,0.937776,0.311725,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
4,0.309859,-4.564744e-01,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,-0.061670,-0.687324,-0.521734,0.937276,-0.120242,0.383659,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.296763,0.600495,0.812705,1.650307,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
5,0.073375,6.336182e-01,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,-0.174865,0.199680,-0.045611,1.617877,-0.120242,1.299326,1.107810,-0.241061,0.789741,1.227585,1.390023,-0.211454,1.527656,0.600495,0.896086,1.650307,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777
6,-0.163109,6.790387e-01,0.360616,-0.795151,-0.517200,0.719786,0.491040,-0.574410,0.632450,-0.288653,-1.139286,-0.596115,-0.948691,0.501875,-0.120242,-0.292145,1.107810,-0.241061,-1.026041,1.227585,-2.288708,-0.211454,-0.934130,-0.951226,0.604253,0.311725,0.032844,-0.432931,-0.251539,-0.359325,10.802446,-0.270208,-0.068692,1.323736,1.360892,0.891994
7,-0.872563,2.248335e-01,-0.043379,1.374795,-0.517200,1.084115,0.975575,0.456019,2.029558,-0.288653,-0.566519,1.433276,1.374993,-0.795163,-0.120242,0.339875,1.107810,-0.241061,0.789741,-0.761621,0.163779,-0.211454,0.296763,0.600495,1.062847,0.311725,0.762732,1.283007,0.156111,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.620891,-0.614439
8,0.073375,6.454645e-16,-0.013513,0.651479,0.381743,0.057371,-0.574938,0.755176,0.910994,-0.090220,-0.795173,0.113032,-0.143941,1.457466,-0.120242,1.093729,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.296763,2.152216,-0.229554,0.311725,0.051559,1.123385,2.375537,3.372372,-0.116339,-0.270208,-0.068692,0.618024,1.730892,0.891994
9,-0.163109,-8.652591e-01,-0.440659,0.651479,-0.517200,-1.333700,-1.689368,-0.574410,-0.973018,-0.288653,0.871057,-0.240402,-0.363889,0.928110,-0.120242,0.492168,-0.819964,-0.241061,0.789741,-0.761621,-1.062465,4.328579,0.912210,2.152216,-1.980549,0.311725,-0.023301,-0.033876,-0.704483,2.995929,-0.116339,-0.270208,-0.068692,-0.087688,-0.859110,0.138777


In [69]:
print(len(number_columns))
print(number_columns)

38
['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [70]:
scaler = preprocessing.StandardScaler()
# scaler_para_most = scaler.fit(train_df[number_columns[:-1]])
# print(scaler_para)

# print(train_df.columns)
# print(number_columns[:-1])
# print()
for column in number_columns[1:-1]:
    print(column)
    for i, df in enumerate(train_test):
        scaler_para = scaler.fit(df[column].as_matrix().reshape(-1, 1))
        if i == 0:
            train_df[column+'_scale'] = scaler.fit_transform(df[column].as_matrix().reshape(-1, 1), scaler_para)
        if i==1:
            test_df[column+'_scale'] = scaler.fit_transform(df[column].as_matrix().reshape(-1, 1), scaler_para)
train_df

MSSubClass
LotFrontage
LotArea
OverallQual
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold


D:\annoconda\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,...,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,MSSubClass_scale,LotFrontage_scale,LotArea_scale,OverallQual_scale,OverallCond_scale,YearBuilt_scale,YearRemodAdd_scale,MasVnrArea_scale,BsmtFinSF1_scale,BsmtFinSF2_scale,BsmtUnfSF_scale,TotalBsmtSF_scale,1stFlrSF_scale,2ndFlrSF_scale,LowQualFinSF_scale,GrLivArea_scale,BsmtFullBath_scale,BsmtHalfBath_scale,FullBath_scale,HalfBath_scale,BedroomAbvGr_scale,KitchenAbvGr_scale,TotRmsAbvGrd_scale,Fireplaces_scale,GarageYrBlt_scale,GarageCars_scale,GarageArea_scale,WoodDeckSF_scale,OpenPorchSF_scale,EnclosedPorch_scale,3SsnPorch_scale,ScreenPorch_scale,PoolArea_scale,MiscVal_scale,MoSold_scale,YrSold_scale
0,1,60,RL,65.000000,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,...,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500,0.073375,-2.293718e-01,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,-0.944591,-0.459303,-0.793434,1.161852,-0.120242,0.370333,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.912210,-0.951226,1.021157,0.311725,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,2,20,RL,80.000000,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,...,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500,-0.872563,4.519361e-01,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,-0.641228,0.466465,0.257140,-0.795163,-0.120242,-0.482512,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.318683,0.600495,-0.104483,0.311725,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,3,60,RL,68.000000,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,...,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500,0.073375,-9.311018e-02,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,-0.301643,-0.313369,-0.627826,1.189351,-0.120242,0.515013,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,-0.318683,0.600495,0.937776,0.311725,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,4,70,RL,60.000000,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,...,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000,0.309859,-4.564744e-01,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,-0.061670,-0.687324,-0.521734,0.937276,-0.120242,0.383659,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.296763,0.600495,0.812705,1.650307,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,5,60,RL,84.000000,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,...,

In [71]:
scaler.mean_

array([ 2007.76970528])

In [72]:
print(len(number_columns))
print(len(str_columns))

38
38


In [73]:
pd.get_dummies(train_df[['MSZoning','Street']])
#发现pd.get_dummies 如果放进去是Series的话，默认没有前缀，如果是DataFrame的话自己加上前缀

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave
0,0,0,0,1,0,0,1
1,0,0,0,1,0,0,1
2,0,0,0,1,0,0,1
3,0,0,0,1,0,0,1
4,0,0,0,1,0,0,1
5,0,0,0,1,0,0,1
6,0,0,0,1,0,0,1
7,0,0,0,1,0,0,1
8,0,0,0,0,1,0,1
9,0,0,0,1,0,0,1


In [74]:
pd.set_option('display.max_columns', 100)

In [75]:
for i, df in enumerate(train_test):
    if i == 0:
        train_str_dummies_columns = pd.get_dummies(df[str_columns])
    if i ==1:
        test_str_dummies_columns = pd.get_dummies(df[str_columns])
print(len(test_str_dummies_columns.columns))
print(len(train_str_dummies_columns.columns))
print(test_str_dummies_columns.columns)
print(train_str_dummies_columns.columns)

train_min_test = list(set(train_str_dummies_columns.columns.tolist()) - set(test_str_dummies_columns.columns.tolist()))
# train_str_dummies_columns.to_csv('train_str.csv')
# test_str_dummies_columns.to_csv('tesr_str.csv')

218
234
Index(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'LotShape_IR1',
       'LotShape_IR2', 'LotShape_IR3',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=218)
Index(['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'LotShape_IR1',
       'LotShape_IR2', 'LotShape_IR3',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=234)


In [76]:
for column in train_min_test: #看一下不一样的原因  train内容更丰富
    aa = column.split('_')[0]
    for df in train_test:
        print(df[aa].value_counts())
    print('\n')

CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Roll          1
Membran       1
Metal         1
ClyTile       1
Name: RoofMatl, dtype: int64
CompShg    1442
Tar&Grv      12
WdShake       4
WdShngl       1
Name: RoofMatl, dtype: int64


AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64
AllPub    1459
Name: Utilities, dtype: int64


CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Roll          1
Membran       1
Metal         1
ClyTile       1
Name: RoofMatl, dtype: int64
CompShg    1442
Tar&Grv      12
WdShake       4
WdShngl       1
Name: RoofMatl, dtype: int64


TA    1392
Fa      48
Gd      14
Po       3
Ex       3
Name: GarageQual, dtype: int64
TA    1371
Fa      76
Gd      10
Po       2
Name: GarageQual, dtype: int64


Norm      1445
Feedr        6
Artery       2
PosN         2
RRNn         2
PosA         1
RRAe         1
RRAn         1
Name: Condition2, dtype: int64
Norm      1444
Feedr        7
PosA         3
Artery       3
PosN         2


In [77]:
train_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       ...
       'GarageArea_scale', 'WoodDeckSF_scale', 'OpenPorchSF_scale',
       'EnclosedPorch_scale', '3SsnPorch_scale', 'ScreenPorch_scale',
       'PoolArea_scale', 'MiscVal_scale', 'MoSold_scale', 'YrSold_scale'],
      dtype='object', length=112)

In [78]:
print(len(train_df.filter(regex='_scale').columns)) #number_columns一共38个，除去ID和saleprice正好是36个，
train_df.filter(regex='_scale').columns

36


Index(['MSSubClass_scale', 'LotFrontage_scale', 'LotArea_scale',
       'OverallQual_scale', 'OverallCond_scale', 'YearBuilt_scale',
       'YearRemodAdd_scale', 'MasVnrArea_scale', 'BsmtFinSF1_scale',
       'BsmtFinSF2_scale', 'BsmtUnfSF_scale', 'TotalBsmtSF_scale',
       '1stFlrSF_scale', '2ndFlrSF_scale', 'LowQualFinSF_scale',
       'GrLivArea_scale', 'BsmtFullBath_scale', 'BsmtHalfBath_scale',
       'FullBath_scale', 'HalfBath_scale', 'BedroomAbvGr_scale',
       'KitchenAbvGr_scale', 'TotRmsAbvGrd_scale', 'Fireplaces_scale',
       'GarageYrBlt_scale', 'GarageCars_scale', 'GarageArea_scale',
       'WoodDeckSF_scale', 'OpenPorchSF_scale', 'EnclosedPorch_scale',
       '3SsnPorch_scale', 'ScreenPorch_scale', 'PoolArea_scale',
       'MiscVal_scale', 'MoSold_scale', 'YrSold_scale'],
      dtype='object')

In [79]:
len(train_str_dummies_columns[test_str_dummies_columns.columns.tolist()].columns)

218

In [80]:
final_train_df = pd.concat([train_df.filter(regex='_scale'), train_str_dummies_columns[test_str_dummies_columns.columns.tolist()]], axis=1)
print(len(final_train_df.columns))
final_train_df.to_csv('./data/final_train_feature.csv')
final_train_df

254


,MSSubClass_scale,LotFrontage_scale,LotArea_scale,OverallQual_scale,OverallCond_scale,YearBuilt_scale,YearRemodAdd_scale,MasVnrArea_scale,BsmtFinSF1_scale,BsmtFinSF2_scale,BsmtUnfSF_scale,TotalBsmtSF_scale,1stFlrSF_scale,2ndFlrSF_scale,LowQualFinSF_scale,GrLivArea_scale,BsmtFullBath_scale,BsmtHalfBath_scale,FullBath_scale,HalfBath_scale,BedroomAbvGr_scale,KitchenAbvGr_scale,TotRmsAbvGrd_scale,Fireplaces_scale,GarageYrBlt_scale,GarageCars_scale,GarageArea_scale,WoodDeckSF_scale,OpenPorchSF_scale,EnclosedPorch_scale,3SsnPorch_scale,ScreenPorch_scale,PoolArea_scale,MiscVal_scale,MoSold_scale,YrSold_scale,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,...,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.073375,-2.293718e-01,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.511418,0.575425,-0.288653,-0.944591,-0.459303,-0.793434,1.161852,-0.120242,0.370333,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,0.912210,-0.951226,1.021157,0.311725,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777,0,0,0,1,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,-0.872563,4.519361e-01,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.574410,1.171992,-0.288653,-0.641228,0.466465,0.257140,-0.795163,-0.120242,-0.482512,-0.819964,3.948809,0.789741,-0.761621,0.163779,-0.211454,-0.318683,0.600495,-0.104483,0.311725,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439,0,0,0,1,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0.073375,-9.311018e-02,0.073480,0.651479,-0.517200,0.984752,0.830215,0.323060,0.092907,-0.288653,-0.301643,-0.313369,-0.627826,1.189351,-0.120242,0.515013,1.107810,-0.241061,0.789741,1.227585,0.163779,-0.211454,-0.318683,0.600495,0.937776,0.311725,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,0.309859,-4.564744e-01,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.574410,-0.499274,-0.288653,-0.061670,-0.687324,-0.521734,0.937276,-0.120242,0.383659,1.107810,-0.241061,-1.026041,-0.761621,0.163779,-0.211454,0.296763,0.600495,0.812705,1.650307,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,0.073375,6.336182e-01,0.375148,1.374795,-0.517200,0.951632,0.733308,1.364570,0.463568,-0.288653,-0.174865,0.199680,-0.045611,1.617877,-0.120242,1.299326,1.107810,-0.241061,0.789741,1.227585,1.390023,-0.211454,1.527656,0.600495,0.896086,1.650307,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0

In [81]:
final_test_df = pd.concat([test_df.filter(regex='_scale'), test_str_dummies_columns], axis=1)
print(len(final_test_df.columns))
# final_test_df.to_csv('./data/final_test_feature.csv')
final_test_df

254


,MSSubClass_scale,LotFrontage_scale,LotArea_scale,OverallQual_scale,OverallCond_scale,YearBuilt_scale,YearRemodAdd_scale,MasVnrArea_scale,BsmtFinSF1_scale,BsmtFinSF2_scale,BsmtUnfSF_scale,TotalBsmtSF_scale,1stFlrSF_scale,2ndFlrSF_scale,LowQualFinSF_scale,GrLivArea_scale,BsmtFullBath_scale,BsmtHalfBath_scale,FullBath_scale,HalfBath_scale,BedroomAbvGr_scale,KitchenAbvGr_scale,TotRmsAbvGrd_scale,Fireplaces_scale,GarageYrBlt_scale,GarageCars_scale,GarageArea_scale,WoodDeckSF_scale,OpenPorchSF_scale,EnclosedPorch_scale,3SsnPorch_scale,ScreenPorch_scale,PoolArea_scale,MiscVal_scale,MoSold_scale,YrSold_scale,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,...,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.874711,0.555587,0.363929,-0.751101,0.400766,-0.340945,-1.072885,-0.570108,0.063295,0.517348,-0.650619,-0.370808,-0.654561,-0.775254,-0.080483,-1.215588,-0.819568,-0.258526,-1.028720,-0.751040,-1.029543,-0.20391,-0.918335,-0.898055,-0.650488,-0.988013,1.185945,0.366678,-0.701628,-0.360738,-0.088827,1.818960,-0.057227,-0.092244,-0.038281,1.713905,0,0,1,0,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,-0.874711,0.604239,0.897861,-0.054877,0.400766,-0.439695,-1.214908,0.041273,1.063392,-0.297903,-0.339378,0.639144,0.433298,-0.775254,-0.080483,-0.323539,-0.819568,-0.258526,-1.028720,1.237648,0.175997,-0.20391,-0.255371,-0.898055,-0.767194,-0.988013,-0.741213,2.347867,-0.178826,-0.360738,-0.088827,-0.301543,-0.057227,19.730438,-0.038281,1.713905,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0.061351,0.263676,0.809646,-0.751101,-0.497418,0.844059,0.678742,-0.570108,0.773254,-0.297903,-0.954994,-0.266876,-0.574165,0.891944,-0.080483,0.294508,-0.819568,-0.258526,0.773083,1.237648,0.175997,-0.20391,-0.255371,0.647066,0.749983,0.301623,0.042559,0.930495,-0.207871,-0.360738,-0.088827,-0.301543,-0.057227,-0.092244,-1.140614,1.713905,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,0.061351,0.458284,0.032064,-0.054877,0.400766,0.876976,0.678742,-0.456889,0.357829,-0.297903,-0.527038,-0.271395,-0.579190,0.837243,-0.080483,0.243004,-0.819568,-0.258526,0.773083,1.237648,0.175997,-0.20391,0.407593,0.647066,0.788885,0.301623,-0.012766,2.089451,-0.178826,-0.360738,-0.088827,-0.301543,-0.057227,-0.092244,-0.038281,1.713905,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1.465443,-1.244533,-0.971808,1.337571,-0.497418,0.679475,0.394694,-0.570108,-0.387298,-0.297903,1.058917,0.528434,0.310192,-0.775254,-0.080483,-0.424487,-0.819568,-0.258526,0.773083,-0.751040,-1.029543,-0.20391,-0.918335,-0.898055,0.555473,0.301623,0.153210,-0.729632,0.489198,-0.360738,-0.088827,2.243060,-0.057227,-0.092244,-1.875504,1.713905,0,0,0,1,0,0,1,1,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,0.

In [82]:
train_df['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
5       143000
6       307000
7       200000
8       129900
9       118000
10      129500
11      345000
12      144000
13      279500
14      157000
15      132000
16      149000
17       90000
18      159000
19      139000
20      325300
21      139400
22      230000
23      129900
24      154000
         ...  
1435    174000
1436    120500
1437    394617
1438    149700
1439    197000
1440    191000
1441    149300
1442    310000
1443    121000
1444    179600
1445    129000
1446    157900
1447    240000
1448    112000
1449     92000
1450    136000
1451    287090
1452    145000
1453     84500
1454    185000
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [83]:
scaler_y = preprocessing.StandardScaler()
scaler_y.fit(train_df['SalePrice'].as_matrix().reshape(-1, 1))
y_scaled = scaler_y.fit_transform(train_df['SalePrice'].as_matrix().reshape(-1, 1), scaler_y)
y_scaled

D:\annoconda\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ..., 
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [84]:
print(scaler_y.var_)
scaler_y.mean_


[  6.30678859e+09]


array([ 180921.19589041])

In [85]:
y_scaled * np.sqrt(scaler_y.var_) + scaler_y.mean_  #这个是后面准备用到的恢复y的公式

array([[ 208500.],
       [ 181500.],
       [ 223500.],
       ..., 
       [ 266500.],
       [ 142125.],
       [ 147500.]])

In [86]:
final_train_df['price'] = y_scaled
print(len(final_train_df.columns))
print(len(final_test_df.columns))

255
254


In [109]:
print(final_train_df.info())
print(final_test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 255 entries, MSSubClass_scale to price
dtypes: float64(37), uint8(218)
memory usage: 732.9 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 254 entries, MSSubClass_scale to SaleCondition_Partial
dtypes: float64(36), uint8(218)
memory usage: 721.0 KB
None


In [87]:
#################################至此 数据预处理完成##############################  y_scaled没有存进csv中###############
#########################################################################################################################

In [107]:
print(final_test_df.as_matrix())
final_train_df.as_matrix()[:, :-1]

[[-0.87471081  0.55558736  0.36392912 ...,  0.          1.          0.        ]
 [-0.87471081  0.60423927  0.89786065 ...,  0.          1.          0.        ]
 [ 0.06135085  0.26367594  0.80964587 ...,  0.          1.          0.        ]
 ..., 
 [-0.87471081  4.44773966  2.05514965 ...,  0.          0.          0.        ]
 [ 0.64638939 -0.3201469   0.12552719 ...,  0.          1.          0.        ]
 [ 0.06135085  0.26367594 -0.03879049 ...,  0.          1.          0.        ]]


array([[ 0.07337496, -0.22937175, -0.20714171, ...,  0.        ,
         1.        ,  0.        ],
       [-0.87256276,  0.4519361 , -0.09188637, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.07337496, -0.09311018,  0.07347998, ...,  0.        ,
         1.        ,  0.        ],
       ..., 
       [ 0.30985939, -0.18395123, -0.14781027, ...,  0.        ,
         1.        ,  0.        ],
       [-0.87256276, -0.09311018, -0.08016039, ...,  0.        ,
         1.        ,  0.        ],
       [-0.87256276,  0.22483348, -0.05811155, ...,  0.        ,
         1.        ,  0.        ]])

In [88]:
#划分数据集###

In [89]:
train_split, test_split = cross_validation.train_test_split(final_train_df, test_size = 0.3)

In [90]:
print(len(test_split.index))
len(train_split.index)


438


1022

In [110]:
#####建立模型#############
# linear_model.
linear_reg_model = linear_model.LinearRegression()
# linear_reg_model.fit(train_split.as_matrix()[:, :-1], y=train_split.as_matrix()[:, -1])
linear_reg_model.fit(final_train_df.as_matrix()[:, :-1], y=final_train_df.as_matrix()[:, -1])
linear_reg_model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [114]:
# predict_y = linear_reg_model.predict(final_test_df.as_matrix())
predict_y = linear_reg_model.predict(test_split.as_matrix()[:, :-1])
print(len(predict_y))
print(linear_reg_model.coef_)
predict_y

438
[ -5.99622877e-02  -1.78805032e-02   6.14947420e-02   1.37555839e-01
   6.49682581e-02   5.73402461e-02   2.38515014e-02   4.91925318e-02
   2.17034253e+10   7.67637675e+09   2.10262367e+10  -2.08757906e+10
  -3.99495675e+10  -4.51103863e+10  -5.02465775e+09   5.43025864e+10
   2.89449692e-02   5.60951233e-03   4.67903614e-02   5.17034531e-03
  -3.20377350e-02  -3.38163376e-02   4.30397391e-02   2.95019150e-02
  -6.04915619e-03   8.95171762e-02  -7.92789459e-03   2.06289291e-02
   2.28428841e-03   7.29835033e-03   1.79262161e-02   2.26068497e-02
   1.55959129e-02  -5.67233562e-03  -1.31174680e-02  -1.20159984e-02
  -1.62662424e+10  -1.62662424e+10  -1.62662424e+10  -1.62662424e+10
  -1.62662424e+10   1.32119516e+10   1.32119516e+10  -2.34826807e+10
  -2.34826807e+10  -2.34826807e+10  -2.34826807e+10   2.57446301e+09
   2.57446301e+09   2.57446301e+09   2.57446301e+09   7.28614062e-01
   7.13997911e+09   7.13997911e+09   7.13997911e+09   7.13997911e+09
   7.13997911e+09  -5.22333002

array([  9.89870071e-01,  -7.12970734e-01,   1.29613876e-01,
         2.03678894e+00,   7.64827728e-02,  -1.38254166e-01,
         1.07303810e+00,  -1.70355225e+00,  -2.09123611e-01,
         7.38706589e-01,   2.58405685e-01,  -4.04981613e-01,
        -2.19377518e-01,  -9.16128159e-01,  -1.37451363e+00,
         7.29530334e-01,   4.83232498e-01,  -8.75517845e-01,
         3.32956314e-01,  -1.65056229e-01,   3.15958023e-01,
         3.60303879e-01,  -1.01501465e+00,  -9.19805527e-01,
        -1.04917526e-01,   1.58742905e-01,   5.26109695e-01,
        -4.47154999e-01,   1.43319511e+00,  -5.58318138e-01,
         3.12482834e-01,   2.15330124e-01,  -3.65638733e-03,
         1.38899422e+00,  -5.10501862e-02,   3.59783173e-02,
        -1.97227478e-01,  -1.10082626e-01,   6.64686203e-01,
        -1.17156792e+00,   2.11884499e-01,  -3.88811111e-01,
         6.69429779e-01,  -3.40223312e-01,  -6.85380936e-01,
         1.51132393e+00,  -8.46075058e-01,  -7.40804672e-01,
         1.88169479e-01,

In [ ]:
print(len(test_split.iloc[:, -1]))
test_split.iloc[:, -1]

In [98]:
print(final_test_df.info())
print('--'*20)
print(final_train_df.info())
print('--'*20)
print(train_split.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 254 entries, MSSubClass_scale to SaleCondition_Partial
dtypes: float64(36), uint8(218)
memory usage: 721.0 KB
None
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 255 entries, MSSubClass_scale to price
dtypes: float64(37), uint8(218)
memory usage: 732.9 KB
None
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 872 to 62
Columns: 255 entries, MSSubClass_scale to price
dtypes: float64(37), uint8(218)
memory usage: 521.0 KB
None


In [95]:
# linear_reg_model = linear_model.LinearRegression()
# linear_reg_model.fit(train_split.as_matrix()[:, :-1], y=train_split.as_matrix()[:, -1])
# predict_y = linear_reg_model.predict(test_split.as_matrix()[:, :-1])



linear_regression = linear_model.LinearRegression()
#linear_reg_model.fit(train_split.as_matrix()[:, :-1], y=train_split.as_matrix()[:, -1])
# linear_reg_model1.fit(X = final_train_df.as_matrix()[:438, :-1], y =final_train_df.as_matrix()[:438, -1])
linear_regression.fit(X = train_split.as_matrix()[:, :-1], y=train_split.as_matrix()[:, -1])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [120]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=6, learning_rate=0.1)
model_xgb.fit(final_train_df.as_matrix()[:, :-1], final_train_df.as_matrix()[:, -1])
model_xgb

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=360,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [123]:
xgb_pre_y = model_xgb.predict(final_test_df.as_matrix())

In [96]:
pre_y = linear_regression.predict(final_test_df.as_matrix())
pre_y

array([  3.75815887e+10,  -3.63489144e+09,   7.78308943e+09, ...,
         6.34979664e+09,   3.18247347e+10,  -1.08683963e+10])

In [124]:
pre_y_rescaled = xgb_pre_y * np.sqrt(scaler_y.var_) + scaler_y.mean_
pre_y_rescaled

array([ 129930.92985729,  153950.33586326,  188817.74669613, ...,
        164572.45473647,  116828.1008195 ,  213170.56530208])

In [127]:
submission = pd.DataFrame(columns=['Id', 'SalePrice'])

In [128]:
submission['Id'] = raw_test_df['Id']

In [129]:
submission['SalePrice'] = pre_y_rescaled

In [132]:
submission.to_csv('./data/submission.csv', index=False)

In [8]:
np.logspace(1, 2.7, 10)

array([  10.        ,   15.44859148,   23.86589787,   36.86945065,
         56.95810811,   87.99225436,  135.93563909,  210.00141557,
        324.42260792,  501.18723363])